### File used to generate unforced trajectories, for comparison between learned and true dynamics

In [ ]:
import torch
import sys
import os
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.cm as cm
import scipy
from datetime import datetime
import json

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
	sys.path.insert(0, module_path)
print(sys.path)

import Series_Elastic_Actuator.Learning.autoencoders_SEA as autoencoders_SEA
import Series_Elastic_Actuator.robot_parameters_SEA as robot_parameters_SEA
import Series_Elastic_Actuator.transforms_SEA as transforms_SEA
import Series_Elastic_Actuator.dynamics_SEA as dynamics_SEA

import Plotting.pendulum_plot as pendulum_plot

import matplotlib
matplotlib.rcParams['font.family']   = 'serif'
matplotlib.rcParams['font.serif']    = ['Times New Roman']
matplotlib.rcParams['mathtext.fontset'] = 'dejavuserif'

from functools import partial



%load_ext autoreload
%autoreload 2


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### Starting parameters

In [ ]:
rp = robot_parameters_SEA.SEA_PARAMETERS


neural_net = False
print(rp)

In [ ]:
t_start = 0.
t_end = 3.
dt = 0.01

model = autoencoders_SEA.Autoencoder_double(rp).to(device)
model_location = 'Models/NN_small_short/NN_202506181821_0.pth'
model.load_state_dict(torch.load(model_location, weights_only=True))

q0min = -torch.pi
q0max = torch.pi

#### Generate $q$-space trajectory

In [ ]:
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [ ]:
def sim_q(t_start, t_end, dt, q_start, q_d_start, u):
	q = q_start
	q_d = q_d_start
	qs_ana = []
	q_ds_ana = []


	n_steps = int((t_end - t_start) / dt)

	for _ in range(n_steps):
		
		M_q, G_q = dynamics_SEA.dynamical_matrices(rp, q.squeeze(0), q_d.squeeze(0))
		A_q = dynamics_SEA.input_matrix()

		tau_q = A_q * u
		q_dd = (torch.pinverse(M_q) @ (tau_q - G_q)).T.to(device)

		q_d = q_d + q_dd * dt
		q = q + q_d * dt
		
		qs_ana.append(q.squeeze(0).detach())
		q_ds_ana.append(q_d.squeeze(0).detach())
	
	return torch.stack(qs_ana,  dim=0), torch.stack(q_ds_ana, dim=0)

#### Generate $\hat{q}$-space trajectory

In [ ]:
timestamp_alt = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

def sim_q_hat(t_start, t_end, dt, q_start, q_d_start, u, model):
	q_nn = q_start
	q_d_nn = q_d_start

	th = model.encoder_vmap(q_nn)
	th_d = (model.jacobian_enc(q_nn) @ q_d_nn.T).T

	q_nn_series, q_d_nn_series = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)

	for t in torch.arange(t_start, t_end, dt):




		q_hat = model.decoder_vmap(th)
		q_d_hat = (model.jacobian_dec(th) @ th_d.T).T
		

		M_q_hat, G_q_hat = dynamics_SEA.dynamical_matrices(rp, q_hat.squeeze(0), q_d_hat.squeeze(0))
		A_q_hat = dynamics_SEA.input_matrix()
		J_h_inv = model.jacobian_dec(th).squeeze(0)
		J_h_inv_trans = torch.transpose(J_h_inv, 0, 1)
		M_th, C_th, G_th = transforms_SEA.transform_dynamical_from_inverse(M_q_hat, G_q_hat, th, th_d, J_h_inv, J_h_inv_trans)
		
		A_th = transforms_SEA.transform_input_matrix_from_inverse_trans(A_q_hat, J_h_inv_trans)
		
		tau_th = A_th * u




		th_dd = (torch.pinverse(M_th) @ (tau_th - C_th @ th_d.T - G_th)).T
		th_d = th_d + th_dd * dt
		th = th + th_d * dt

		
		q_nn = model.decoder_vmap(th)
		q_d_nn = (model.jacobian_dec(th) @ th_d.T).T

		q_nn_series = torch.cat((q_nn_series, q_nn.detach()), dim = 0)
		q_d_nn_series = torch.cat((q_d_nn_series, q_d_nn.detach()), dim = 0)

	return q_nn_series, q_d_nn_series


In [ ]:
def check_q_outside_index(q_series, q0_min, q0_max):
	outside = (q_series[:, 0] > q0_max) | (q_series[:, 0] < q0_min)

	if outside.any():
		idxs = torch.nonzero(outside, as_tuple=True)[0]
		outside_idx = idxs[0].item()
		return outside_idx
	else:
		return q_series.size(0)

In [ ]:
def create_data_lists(qs_start, q_ds_start, us, loaded_data = None, print_iter = None, save_path = None):


	if loaded_data is None:
		q_ana_series_list, q_d_ana_series_list = [], []
		q_nn_series_list, q_d_nn_series_list = [], []
	else:
		q_ana_series_list, q_d_ana_series_list = loaded_data["q_ana_sl"], loaded_data["q_d_ana_sl"]
		q_nn_series_list, q_d_nn_series_list = loaded_data["q_nn_sl"], loaded_data["q_d_nn_sl"]


	for i, (q_start, q_d_start, u) in enumerate(zip(qs_start, q_ds_start, us)):
		if print_iter is not None:
			if i % print_iter == 0:
				print("sim nr:", i)


		# Simulate ANALYTIC trajectory
		q_ana_series, q_d_ana_series = sim_q(t_start, t_end, dt, q_start.clone(), q_d_start.clone(), u)
		


		# Check for exclusion criteria, trim based on most stringent one
		q_outside_idx = check_q_outside_index(q_ana_series, q0min, q0max)
		#save_idx = min(too_long_idx, q_outside_idx, too_short_idx)
		save_idx = q_outside_idx

		if save_idx > 0:

			# Only simulate NEURAL NETWORK trajectory if analytic makes it the whole way
			q_nn_series, q_d_nn_series = sim_q_hat(t_start, t_end, dt, q_start.clone(), q_d_start.clone(), u, model)

			q_ana_series_list.append(q_ana_series[:save_idx])
			q_d_ana_series_list.append(q_d_ana_series[:save_idx])

			q_nn_series_list.append(q_nn_series[:save_idx])
			q_d_nn_series_list.append(q_d_nn_series[:save_idx])

		if save_path is not None:
			data_to_save = {
				"q_ana_sl": q_ana_series_list,
				"q_d_ana_sl": q_d_ana_series_list,
				"q_nn_sl": q_nn_series_list,
				"q_d_nn_sl": q_d_nn_series_list,
			}
			torch.save(data_to_save, save_path)
	
	return q_ana_series_list, q_d_ana_series_list, q_nn_series_list, q_d_nn_series_list

In [ ]:
def create_plot_datasets(q_ana_series, q_nn_series):
	datasets_q = [
		{
			"name": "Learned",
			"values": q_nn_series.cpu().detach().numpy(),
			"color": "tab:orange"
		},
		{
			"name": "Ground truth",
			"values": q_ana_series.cpu().detach().numpy(),
			"color": "tab:blue"
		}
	]



	# Common labels for the plots.
	name_q = "q trajectory"
	t_series = torch.arange(0, t_end, dt)
	t_series = t_series[:q_ana_series.size(0)]

	# Create an instance of ErrorPlotter.
	ep = pendulum_plot.Error_plotter(rp)

	# Prepare plot datasets for each column.
	# Each call groups a set of datasets to be drawn in one subplot column.
	q_plot_dataset = ep.create_plot_dataset(t=t_series, datasets=datasets_q, reference=None, name=name_q)


	plot_datasets = [q_plot_dataset]
	plot_colormaps = ["Oranges", "Blues", "Greens"]

	return plot_datasets, plot_colormaps

In [ ]:

def make_error_plots(plot_datasets, save_dir, ep):
	file_name = "Error plot.png"
	file_counter = 0

	output_path = os.path.join(save_dir, file_name)

	# Pass the list of columns (plot_dataset objects) to plot_multi.
	ep.plot_multi(plot_datasets=plot_datasets, save_path=output_path, axes_names = ["q", "th"])




#### Plot $q$-space error

In [ ]:
from matplotlib.lines import Line2D
def plot_q_trajs(q_dataset, plot_colormaps):
	fig, ax = plt.subplots(figsize=(6,4))
	scs = []
	for i, cmap in zip(range(2), plot_colormaps):
		t = q_dataset["x"]                      # shape (N,)
		q1 = q_dataset["data"][i]["y1"]         # shape (N,)
		q2 = q_dataset["data"][i]["y2"]         # shape (N,)
		sc = ax.scatter(q1, q2, c=t, cmap=cmap, s=20)

		# optionally connect points in order
		ax.plot(q1, q2, lw=0.5, color='gray', alpha=0.5)
		scs.append(sc)

	# labels and colorbar
	ax.set_xlabel('$q_0$')
	ax.set_ylabel('$q_1$')
	ax.grid()

	cbar1 = fig.colorbar(
		scs[0], ax=ax, pad=-0.01, fraction=0.057
	)
	cbar1.set_label('time')

	cbar2 = fig.colorbar(
		scs[1], ax=ax, pad=0.02, fraction=0.057, ticks=[]
	)

	labels = [q_dataset["data"][i]["name"] for i in range(2)]
	proxies = []
	for cmap, label in zip(plot_colormaps, labels):
		color = plt.get_cmap(cmap)(0.7)
		proxies.append(Line2D([0], [0], marker='o', 
							  color='w',
							  markerfacecolor=color,
							  markersize=8, 
							  linestyle='',
							  label=label))

	ax.legend(handles=proxies, title="Trajectories")

	ax.set_title('Trajectory in $(q_0,q_1)$ colored by time')
	plt.tight_layout()
	plt.show()

In [ ]:
n_sims = 20
print_iter = max(torch.floor(torch.tensor(n_sims)/100).item(), 1)


q0min, q0max = -torch.pi/2, torch.pi/2
q1min, q1max = 0, torch.pi
q0s = torch.rand(n_sims) * (q0max - q0min) + q0min
q1s = torch.rand(n_sims) * (q1max - q1min) + q1min
qs_start = torch.stack((q0s, q1s), dim=1).requires_grad_().to(device)



q_d0min, q_d0max = -1, 1
q_d1min, q_d1max = -1, 1
q_d0s = torch.rand(n_sims) * (q_d0max - q_d0min) + q_d0min
q_d1s = torch.rand(n_sims) * (q_d1max - q_d1min) + q_d1min 
q_ds_start = torch.stack((q0s, q1s), dim=1).requires_grad_().to(device)


print(qs_start.size())

qs_start = qs_start.unsqueeze(1)
q_ds_start = q_ds_start.unsqueeze(1)

us = -5 + 10 * torch.rand(n_sims)

#### Load the data (in case you want to append to existing trials)

In [ ]:
load_data = False
loaded_data_dir = "Performance_Sims/Optimal"
load_num = 0
if load_data:
	loaded_data = torch.load(loaded_data_dir+"/simulations_data_" + str(load_num) + ".pt")
	print("Loading simulation data from:", loaded_data_dir)
else:
	loaded_data = None
	print("Not loading simulation data.")


In [ ]:
save_dir = "Performance_Sims/Optimal"
save_path = save_dir + "/simulations_data_0.pt"
file_iter = 1
while os.path.isfile(save_path):
    save_path = save_path[:-4] + str(file_iter) + ".pt"
    file_iter += 1

q_ana_sl, q_d_ana_sl, q_nn_sl, q_d_nn_sl = create_data_lists(qs_start, q_ds_start, us, loaded_data = loaded_data, 
																				  print_iter = print_iter, save_path=save_path)

#### Save the data (also happens during generation)

In [ ]:
os.makedirs(save_dir, exist_ok=True)

data_to_save = {
	"q_ana_sl": q_ana_sl,
	"q_d_ana_sl": q_d_ana_sl,	
	"q_nn_sl": q_nn_sl,
	"q_d_nn_sl": q_d_nn_sl,
}

torch.save(data_to_save, save_dir + "/simulations_data_" + str(file_iter) + ".pt")

#### Load the data (in case you're running without generating)

In [ ]:
load_data_dir = "Performance_Sims/Optimal"
load_num = 1
loaded_data = torch.load(load_data_dir + "/simulations_data_" + str(load_num) + ".pt")

q_ana_sl = loaded_data["q_ana_sl"]
q_d_ana_sl = loaded_data["q_d_ana_sl"]
q_nn_sl = loaded_data["q_nn_sl"]
q_d_nn_sl = loaded_data["q_d_nn_sl"]

print("Loaded model containing",  len(q_ana_sl), "simulations.")

In [ ]:
q_error_list = []
plotter_counter = 0
for i, (qas, qdas, qns, qdns) in enumerate(zip(q_ana_sl, q_d_ana_sl, q_nn_sl, q_d_nn_sl)):

	q_error_series = qas - qns
	q_d_error_series = qdas - qdns
	plot_datasets, plot_colormaps = create_plot_datasets(qas, qns)
	q_plot_dataset = plot_datasets[0]
	if qas.size(0) == 300 or True:
		q_error_list.append(q_error_series)

		if plotter_counter < 5:
			plot_q_trajs(q_plot_dataset, plot_colormaps)
			plotter_counter += 1




In [ ]:

for i in range(20):
	print(q_ana_sl[i].size())


In [ ]:
# Number of trials that made it to the end
print(len(q_error_list))

In [ ]:
# Transform the q_error_list into a pytorch tensor
q_error_list_full = torch.nn.utils.rnn.pad_sequence(q_error_list, batch_first = True, padding_value=float("nan"))

# Calculate the sample mean
q_error_mean = torch.nanmean(q_error_list_full, dim=0)

# Calculate the rmse error manually
q_error_square = torch.square(q_error_list_full)
q_error_mse = torch.nanmean(q_error_square, dim=0)
q_error_rmse = torch.sqrt(q_error_mse)

# Create mask which is True where index is NOT nan
nan_mask = ~torch.isnan(q_error_list_full)
# Log how many of the trials are not nan at each time step
counts = nan_mask.sum(dim=(0))[:,0]

# Calculate the sample standard deviation manually
q_error_diff = q_error_list_full - q_error_mean
q_error_sqdiff = torch.square(q_error_diff)
q_error_sumsqdiff = torch.nansum(q_error_sqdiff, dim=0)
q_error_var = q_error_sumsqdiff / (counts-1).unsqueeze(1)
q_error_std = torch.sqrt(q_error_var)

In [ ]:
def calc_rmse(error_list):
	error_list_full = torch.nn.utils.rnn.pad_sequence(error_list, batch_first = True, padding_value=float("nan"))
	error_square = torch.square(error_list_full)
	error_mse = torch.nanmean(error_square, dim=0)
	error_rmse = torch.sqrt(error_mse)
	return error_rmse

def calc_std(error_list):
	error_list_full = torch.nn.utils.rnn.pad_sequence(error_list, batch_first = True, padding_value=float("nan"))
	error_mean = torch.nanmean(error_list_full, dim=0)
	nan_mask = ~torch.isnan(error_list_full)
	counts = nan_mask.sum(dim=(0))[:,0]
	error_diff = error_list_full - error_mean
	error_sqdiff = torch.square(error_diff)
	error_sumsqdiff = torch.nansum(error_sqdiff, dim=0)
	error_var = error_sumsqdiff / (counts-1).unsqueeze(1)
	error_std = torch.sqrt(error_var)
	return error_std

In [ ]:
def plot_error_std(error_rmse, error_std, ylim, y1label, y2label, yaxislabel, title_string, save_dir, file_name):
	t = torch.arange(error_rmse.size(0))/100

	y = error_rmse.cpu().detach().numpy()
	ystd = error_std.cpu().detach().numpy()

	fig, ax = plt.subplots(figsize=(5, 2.5))

	# Plot each column with shaded std area
	colors = ["C0", "C1"]
	for i, label in enumerate([y1label, y2label]):
		ax.plot(t, y[:, i], label=label, color = colors[i], linewidth = 2.5)
		ax.fill_between(
			t,
			(y[:, i] - ystd[:, i]),
			(y[:, i] + ystd[:, i]),
			alpha=0.1,
			color = colors[i]
		)
		ax.plot(t, (y[:, i] - ystd[:, i]), linestyle="--", color = colors[i], alpha = 0.5)
		ax.plot(t, (y[:, i] + ystd[:, i]), linestyle="--", color = colors[i], alpha = 0.5)

	ax.set_xlabel('time (s)')
	ax.set_ylabel(yaxislabel)
	ax.set_title('SEA learned dynamics prediction error ' + title_string)
	#ax.set_ylim(ylim)
	ax.set_xlim((0, error_rmse.size(0)/100))
	ax.legend()
	ax.grid()
	plt.tight_layout()
	plt.savefig(save_dir + file_name +".pdf", format="pdf")
	plt.show()
	


In [ ]:

plot_error_std(q_error_rmse, q_error_std, (0, 0.2), r"$\theta_0$", r"$\theta_1$", "RMSE (rad)", "in original angles " + r"$\theta$", save_dir, "/Dynamics_prediction_error_q")